In [ ]:
from requests import Session
import MatifyAPI
import io
sess = Session()
categories = MatifyAPI.getCategories(sess, verbose=False)

In [ ]:
#Request products from all sub category
allProducts = []
nTotalProducts = 0
for categoryId, categoryName, subCategories in categories:
    print "-------------------------------------------"
    products = MatifyAPI.getProducts (sess, categoryId, categoryName, verbose=True)
    nTotalProducts += len(products)
    allProducts.append([categoryName, products])
    for subCategoryID, subCategoryName, _ in subCategories:
        products = MatifyAPI.getProducts (sess, subCategoryID, subCategoryName, verbose=True)
        nTotalProducts += len(products)
        allProducts.append([subCategoryName, products])

print "Total Products: "  + str(nTotalProducts)

In [ ]:
productWithImages = MatifyAPI.filterProductWithImage (allProducts, verbose=True)

In [ ]:
#Download all product images to the corresponding folder
from MatifyDataset import MatifyDataset
matifyDataset = MatifyDataset(datasetDir = 'MatifyDataset/', leastExamples = 15, nShards = 1, 
                              nValidations = 20, randomSeed = 0, verbose = True);
#matifyDataset.download();
matifyDataset.convertToTfrecord()

In [ ]:
import tensorflow as tf
import math
import sys
sys.path.append(os.path.abspath("/notebooks/AutoParser/squeezenet/models/slim/"))
from datasets import dataset_utils
_NUM_SHARDS = 1
class ImageReader(object):
    """Helper class that provides TensorFlow image coding utilities."""

    def __init__(self):
        # Initializes function that decodes RGB JPEG data.
        self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

    def read_image_dims(self, sess, image_data):
        image = self.decode_jpeg(sess, image_data)
        return image.shape[0], image.shape[1]

    def decode_jpeg(self, sess, image_data):
        image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

def _get_dataset_filename(dataset_dir, split_name, shard_id):
    output_filename = 'matify_%s_%05d-of-%05d.tfrecord' % (
        split_name, shard_id, _NUM_SHARDS)
    return os.path.join(dataset_dir, output_filename)

def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir):
    """Converts the given filenames to a TFRecord dataset.
    Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
    """
    assert split_name in ['train', 'validation']

    num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

    with tf.Graph().as_default():
        image_reader = ImageReader()

        with tf.Session('') as sess:

            for shard_id in range(_NUM_SHARDS):
                output_filename = _get_dataset_filename(dataset_dir, split_name, shard_id)

                with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
                    start_ndx = shard_id * num_per_shard
                    end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
                    for i in range(start_ndx, end_ndx):
                        sys.stdout.write('\r>> Converting image %d/%d shard %d' % (i+1, len(filenames), shard_id))
                        sys.stdout.flush()

                        # Read the filename:
                        image_data = tf.gfile.FastGFile(filenames[i], 'r').read()
                        height, width = image_reader.read_image_dims(sess, image_data)

                        class_name = os.path.basename(os.path.dirname(filenames[i]))
                        class_id = class_names_to_ids[class_name]

                        example = dataset_utils.image_to_tfexample(
                            image_data, 'jpg', height, width, class_id)
                        tfrecord_writer.write(example.SerializeToString())

    sys.stdout.write('\n')
    sys.stdout.flush()


In [ ]:
# First, convert the training and validation sets.
_convert_dataset('train', training_filenames, class_names_to_ids,
               datasetDir)
_convert_dataset('validation', validation_filenames, class_names_to_ids,
               datasetDir)

# Finally, write the labels file:
labels_to_class_names = dict(zip(range(len(class_names)), class_names))
dataset_utils.write_label_file(labels_to_class_names, datasetDir)

print('\nFinished converting the Matify dataset!')
